In [1]:
!pip install flask flask-ngrok ibm-db pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 25.4 MB/s eta 0:00:00


In [2]:
from flask import Flask, render_template, request, redirect, url_for
from flask_ngrok import run_with_ngrok
import ibm_db
import pandas as pd
import os

In [3]:
# IBM DB2 connection setup
dsn_hostname = os.getenv('DB_HOST', '764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud')
dsn_uid = os.getenv('DB_UID', 'rrp03094')
dsn_pwd = os.getenv('DB_PWD', 'yK5spscaxtWuPnE2')
dsn_port = os.getenv('DB_PORT', '32536')
dsn_database = "bludb"
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_protocol = "TCPIP"
dsn_security = "SSL"

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)

In [4]:
# Fetch students from DB2
def fetch_students():
    try:
        conn = ibm_db.connect(dsn, "", "")
        query = "SELECT * FROM STUDENTS"
        stmt = ibm_db.exec_immediate(conn, query)
        rows = []
        result = ibm_db.fetch_assoc(stmt)
        while result:
            rows.append(result)
            result = ibm_db.fetch_assoc(stmt)
        df = pd.DataFrame(rows)
        return df
    except Exception as e:
        print("Error fetching students from DB2: ", str(e))
        return None
    finally:
        if 'conn' in locals():
            ibm_db.close(conn)

In [5]:
# Add a student to DB2
def add_student(name, birth_date, student_id, major):
    try:
        conn = ibm_db.connect(dsn, "", "")
        query = """
        INSERT INTO STUDENTS ("TEN_SINH_VIEN", "NGAY_SINH", "MA_SV", "CHUYEN_NGANH")
        VALUES (?, ?, ?, ?)
        """
        stmt = ibm_db.prepare(conn, query)
        ibm_db.bind_param(stmt, 1, name)
        ibm_db.bind_param(stmt, 2, birth_date)
        ibm_db.bind_param(stmt, 3, student_id)
        ibm_db.bind_param(stmt, 4, major)
        ibm_db.execute(stmt)
    except Exception as e:
        print("Error adding student to DB2: ", str(e))
    finally:
      pass
    return redirect(url_for('index'))

In [6]:
# Delete a student from DB2
def delete_student(student_id):
    try:
        conn = ibm_db.connect(dsn, "", "")
        query = "DELETE FROM STUDENTS WHERE \"MA_SV\" = ?"
        stmt = ibm_db.prepare(conn, query)
        ibm_db.bind_param(stmt, 1, student_id)
        ibm_db.execute(stmt)
    except Exception as e:
        print("Error deleting student from DB2: ", str(e))
    finally:
        if 'conn' in locals():
            ibm_db.close(conn)

In [7]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [8]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://b1w56t1fojk-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [9]:
app = Flask(__name__)


In [10]:
# Main route to display student data and handle add/delete requests
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'add' in request.form:
            name = request.form['name']
            birth_date = request.form['birth_date']
            student_id = request.form['student_id']
            major = request.form['major']
            # Add student to DB2
            add_student(name, birth_date, student_id, major)
        elif 'delete' in request.form:
            student_id = request.form['student_id']
            # Delete student from DB2
            delete_student(student_id)

    # Fetch updated student list from DB2 and display
    students = fetch_students()
    return render_template('index.html', students=students)

# Run the Flask app
if __name__ == '__main__':
    app.run(port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 13:29:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 13:29:57] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 13:30:49] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 13:30:51] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 13:30:59] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 13:31:00] "GET /favicon.ico HTTP/1.1" 404 -
